# Script to Consolidate Data and Create a Day Count

In [58]:
import pandas as pd
import glob

In [59]:
path = "D:/Dropbox/Data/R Shiny/20-03-20 Covid-19"

## Combine all the files together

In [60]:
all_files = glob.glob(path + "/csse_covid_19_daily_reports/*.csv")

li = []

# for files in folder, create a dataframe out of them
for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    # create a date field that is generated from the file name
    df['date'] = filename.split('\\')[-1].split('.')[0]
    # add data to list
    li.append(df)

# concatinate all data (union) into final dataframe    
frame = pd.concat(li, axis=0, ignore_index=True,sort=False)

## Aggregate the numbers on a country level

In [61]:
# aggregate numbers by country
country_df = frame.groupby(['Country/Region','date']).sum().reset_index()

In [62]:
country_list = []

# split the dataframe by country
for country, country_df in country_df.groupby('Country/Region'):
    # Add a day field and number it chronologically
    country_df = country_df.reset_index()
    country_df['Days'] = country_df.index + 1
    
    # Get deltas for deaths, recovered, and then new cases using pd.diff() function
    country_df['D_delta'] = country_df['Deaths'].diff()
    country_df['R_delta'] = country_df['Recovered'].diff()
    country_df['C_delta'] = country_df['Confirmed'].diff()
    
    
    # add data to list
    country_list.append(country_df)
    
# concatinate all data (union) into final dataframe    
frame2 = pd.concat(country_list, axis=0, ignore_index=True,sort=False)

## Cleaning the data

In [63]:
# delete newly created fields
del frame2['index']

# rearrange fields for easier reading
final_table = frame2[['date','Days','Country/Region','Confirmed','C_delta','Deaths','D_delta','Recovered','R_delta']]
# Get active cases for the day
final_table['Active_Cases'] = final_table['Confirmed'] - final_table['Deaths'] - final_table['Recovered']

D:\Users\Shafquat\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [64]:
# for row in dataframe
for i, row in final_table.iterrows():
    # if Days == 1
    if (row['Days'] == 1):
        # C_delta == Confirmed, D_delta == Deaths, R_delta == Recovered
        final_table.at[i,'C_delta'] = row['Confirmed']
        final_table.at[i,'D_delta'] = row['Deaths']
        final_table.at[i,'R_delta'] = row['Recovered']

In [65]:
final_table[final_table['Country/Region'] == 'US']

,date,Days,Country/Region,Confirmed,C_delta,Deaths,D_delta,Recovered,R_delta,Active_Cases
3324,01-22-2020,1,US,1.0,1.0,0.0,0.0,0.0,0.0,1.0
3325,01-23-2020,2,US,1.0,0.0,0.0,0.0,0.0,0.0,1.0
3326,01-24-2020,3,US,2.0,1.0,0.0,0.0,0.0,0.0,2.0
3327,01-25-2020,4,US,2.0,0.0,0.0,0.0,0.0,0.0,2.0
3328,01-26-2020,5,US,5.0,3.0,0.0,0.0,0.0,0.0,5.0
3329,01-27-2020,6,US,5.0,0.0,0.0,0.0,0.0,0.0,5.0
3330,01-28-2020,7,US,5.0,0.0,0.0,0.0,0.0,0.0,5.0
3331,01-29-2020,8,US,5.0,0.0,0.0,0.0,0.0,0.0,5.0
3332,01-30-2020,9,US,5.0,0.0,0.0,0.0,0.0,0.0,5.0
3333,01-31-2020,10,US,6.0,1.0,0.0,0.0,0.0,0.0,6.0


In [66]:
final_table.to_csv(path + "/transformed.csv", index=False)